### Big Data Project

Approach 2: Converting the Hindi Dataset to a standard English dataset and applying Logistic Regression model on it. 

In [2]:
'''
Why are we using this approach?
The unique factor that our project serves is that, it can detect fake news among different native languages, but the validated datasets are limited in very few languages. So, if we get a good accuracy using this model, different datasets in native languages (if developed in future for example; Gujarati, Punjabi, Marathi, Kannada) can directly be converted to a standard language and can be used in this model. 
'''


'\nWhy are we using this approach?\nThe unique factor that our project serves is that, it can detect fake news among different native languages, but the validated datasets are limited in very few languages. So, if we get a good accuracy using this model, different datasets in native languages (if developed in future for example; Gujarati, Marathi, Kannada) can directly be converted to a standard language and can be used in this model. \n'

In [2]:
# Importing the libraries
import pandas as pd
import json                # Importing the json library for importing json files    
import re                  # Importing re library for manipulation with RegEx i.e. regular expressions.
import numpy as np
from numpy import unicode

from googletrans import Translator
from tqdm import tqdm 


import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

C:\Users\bhavy\AppData\Local\Temp\ipykernel_22612\74823216.py:6: DeprecationWarning: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing this will not modify any behaviour and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import unicode


Data Loading


In [4]:
with open('Data_Approach2\\fake_news.json', encoding="utf8") as f:
  fake_news = json.load(f)
  #json to dataframe
  fake_news = pd.json_normalize(fake_news,'fake_news')
  f.close()

with open('Data_Approach2\\valid_news.json', encoding="utf8") as f:
  true_news = json.load(f)
  #json to dataframe
  true_news = pd.json_normalize(true_news,'true_news')
  f.close()

Data Cleaning

In [5]:
# Changing the column name so that it matches the fake news.
true_news = true_news.rename(columns={'short_desc': 'short_description'})

In [6]:
# Counting the NULL values in the dataframes
fake_news.isnull().sum(axis = 0)

url                  0
short_description    0
full_title           5
long_description     9
dtype: int64

In [7]:
# Counting the NULL values in the dataframes
true_news.isnull().sum(axis = 0)

name                   0
url                  120
short_description    133
full_title           450
long_description     450
dtype: int64

In [8]:
# Deleting the URLs, names, titles, and long description of the news as we don't require them
true_news=true_news.drop(['name','url', 'full_title','long_description',],axis=1)
fake_news=fake_news.drop(['url','full_title','long_description',],axis=1)

In [9]:
print("Final Dataset of Fake news with only short_description")
fake_news

Final Dataset of Fake news with only short_description


,short_description
0,बूम ने पाया कि इमरान खान भारत सरकार की आलोचना ...
1,सिख समुदाय के कुछ लोगों को हिंदी साइन बोर्ड पर...
2,सोशल मीडिया प्लेटफ़ॉर्म फ़ेसबुक और ट्विटर पर इस ...
3,दावा है कि भाजपा के नेताओं ने माना है कि सरकार...
4,मीडिया आउटलेट्स और वायर एजेंसी ने गलत तरीके से...
...,...
1245,भारतीय पहलवान कविता देवी और बुल बुल के बीच हुए...
1246,बूम से बात करते हुए एसबीआई अधिकारी ने बताया कि...
1247,बीजेपी सांसद परेश रावल एक बार फिर अपने फेक न्य...
1248,मार्च से लेकर मई महीने में अचानक से एक्टिव हुई...


In [10]:
print("Final Dataset of Valid news with only short_description")
true_news

Final Dataset of Valid news with only short_description


,short_description
0,भारत ने ऑस्ट्रेलिया में टेस्ट सिरीज़ में ऐतिहा...
1,ब्रिस्बेन टेस्ट में भारत की दीवार कहे जाने वाल...
2,पाकिस्तानी टीवी प्रेजेंटर सैय्यद इक़रारुल हसन ...
3,किसी से पूछा जाए कि दिल्ली में पहली गणतंत्र दि...
4,वेब सिरीज़ तांडव को लेकर जारी विवाद पर आज का क...
...,...
888,ये मिसाइल 400 किलोमीटर से ज़्यादा मार कर सकती है.
889,"आकर्षण का केंद्र वो शक्तिशाली पहलवान होते थे, ..."
890,"बिहार के जातिगत समीकरणों की बात बहुत होती है, ..."
891,एनआईए ने 83 साल के फ़ादर स्टेन स्वामी को गिरफ़...


In [11]:
# Deleting the rows with empty values
fake_news=fake_news.dropna()
true_news=true_news.dropna()

In [12]:
# Replacing all the special symbols by an empty string or a null value.
# Basically removing all the special characters.

for i in list(true_news):
  true_news[i]=true_news[i].str.replace('|', '')
  true_news[i]=true_news[i].str.replace('?', '')
  true_news[i]=true_news[i].str.replace(':', '')
  true_news[i]=true_news[i].str.replace(';', '')
  true_news[i]=true_news[i].str.replace("'", '')
  true_news[i]=true_news[i].str.replace('"', '')
  true_news[i]=true_news[i].str.replace(',', '')
  true_news[i]=true_news[i].str.replace('.', '')
  true_news[i]=true_news[i].str.replace('(', '')
  true_news[i]=true_news[i].str.replace(')', '')
  true_news[i]=true_news[i].str.replace('\n', '')
  true_news[i]=true_news[i].str.replace('&', '')

for i in list(fake_news):
  fake_news[i]=fake_news[i].str.replace('|', '')
  fake_news[i]=fake_news[i].str.replace('?', '')
  fake_news[i]=fake_news[i].str.replace(':', '')
  fake_news[i]=fake_news[i].str.replace(';', '')
  fake_news[i]=fake_news[i].str.replace("'", '')
  fake_news[i]=fake_news[i].str.replace('"', '')
  fake_news[i]=fake_news[i].str.replace(',', '')
  fake_news[i]=fake_news[i].str.replace('.', '')
  fake_news[i]=fake_news[i].str.replace('(', '')
  fake_news[i]=fake_news[i].str.replace(')', '')
  fake_news[i]=fake_news[i].str.replace('\n', '')
  fake_news[i]=fake_news[i].str.replace('&', '')
  fake_news[i]=fake_news[i].str.replace('।', '')

C:\Users\bhavy\AppData\Local\Temp\ipykernel_11188\2706083666.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  true_news[i]=true_news[i].str.replace('|', '')
C:\Users\bhavy\AppData\Local\Temp\ipykernel_11188\2706083666.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_news[i]=true_news[i].str.replace('|', '')
C:\Users\bhavy\AppData\Local\Temp\ipykernel_11188\2706083666.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  true_ne

In [13]:
fake_news

,short_description
0,बूम ने पाया कि इमरान खान भारत सरकार की आलोचना ...
1,सिख समुदाय के कुछ लोगों को हिंदी साइन बोर्ड पर...
2,सोशल मीडिया प्लेटफ़ॉर्म फ़ेसबुक और ट्विटर पर इस ...
3,दावा है कि भाजपा के नेताओं ने माना है कि सरकार...
4,मीडिया आउटलेट्स और वायर एजेंसी ने गलत तरीके से...
...,...
1245,भारतीय पहलवान कविता देवी और बुल बुल के बीच हुए...
1246,बूम से बात करते हुए एसबीआई अधिकारी ने बताया कि...
1247,बीजेपी सांसद परेश रावल एक बार फिर अपने फेक न्य...
1248,मार्च से लेकर मई महीने में अचानक से एक्टिव हुई...


In [14]:
true_news

,short_description
0,भारत ने ऑस्ट्रेलिया में टेस्ट सिरीज़ में ऐतिहा...
1,ब्रिस्बेन टेस्ट में भारत की दीवार कहे जाने वाल...
2,पाकिस्तानी टीवी प्रेजेंटर सैय्यद इक़रारुल हसन ...
3,किसी से पूछा जाए कि दिल्ली में पहली गणतंत्र दि...
4,वेब सिरीज़ तांडव को लेकर जारी विवाद पर आज का क...
...,...
887,चालक ने कार रोकने के बजाय उसे चलाना जारी रखा औ...
888,ये मिसाइल 400 किलोमीटर से ज़्यादा मार कर सकती है
889,आकर्षण का केंद्र वो शक्तिशाली पहलवान होते थे ज...
890,बिहार के जातिगत समीकरणों की बात बहुत होती है ल...


In [15]:
# Converting the dataframe to a list in order to convert it in English language.

df_fake = pd.DataFrame(fake_news)
news_fake = df_fake['short_description'].tolist()


df_fake = pd.DataFrame(true_news)
news_true = df_fake['short_description'].tolist()

In [16]:
news_fake

['बूम ने पाया कि इमरान खान भारत सरकार की आलोचना कर रहे थे और वर्तमान शासन को अधिनायकवादी मुस्लिम विरोधी और पाकिस्तान',
 'सिख समुदाय के कुछ लोगों को हिंदी साइन बोर्ड पर कालिख पोतते दिखाती तस्वीरों का सेट और एक वीडियो वायरल हो रहा है',
 'सोशल मीडिया प्लेटफ़ॉर्म फ़ेसबुक और ट्विटर पर इस दावे के साथ बड़ी संख्या में तस्वीर के साथ पोस्ट शेयर किया जा रहा है',
 'दावा है कि भाजपा के नेताओं ने माना है कि सरकार जनता के साथ गलत कर रही है ',
 'मीडिया आउटलेट्स और वायर एजेंसी ने गलत तरीके से दावा किया कि पाकिस्तान के पूर्व राजनयिक ने बालाकोट में हुई मौतों की',
 'यूज़र्स तस्वीर शेयर करते हुए दावा कर रहे हैं कि यह तस्वीर पुरानी दिल्ली रेलवे स्टेशन के प्लेटफ़ॉर्म पर बनी एक मस्जिद की',
 'बूम ने पाया कि मूल तस्वीर साल 2014 से है जब अमित शाह प्रधानमंत्री नरेंद्र मोदी को बधाई दे रहे थे',
 'बूम ने पाया कि वायरल तस्वीर साल 2018 में उत्तर प्रदेश के उन्नाव में हुई एक हत्या की है',
 'दावा है कि कोविड-19 के टीके लिंग पर लगाने से वायरस से जल्द राहत मिलेगी ',
 'दावा वायरल है कि ब्रिटेन के प्रधानमंत्री ने किसान आंदोलन के 

In [17]:
news_true

['भारत ने ऑस्ट्रेलिया में टेस्ट सिरीज़ में ऐतिहासिक जीत दर्ज की चौथे और निर्णायक मैच में नौजवान गेंदबाज़ मोहम्मद सिराज ने अहम भूमिका निभाई',
 'ब्रिस्बेन टेस्ट में भारत की दीवार कहे जाने वाले चेतेश्वर पुजारा ने अर्धशतक जमाया पारी के दौरान वो चोटिल भी हुए और ऑस्ट्रेलियाई गेंदबाज़ों की कई गेंदे उनके शरीर पर लगीं',
 'पाकिस्तानी टीवी प्रेजेंटर सैय्यद इक़रारुल हसन अपने कुछ ट्वीट के कारण निशाने पर आ गए उन्हें देशद्रोही तक कहा जाने लगा',
 'किसी से पूछा जाए कि दिल्ली में पहली गणतंत्र दिवस परेड कहां हुई थी तो इसका जवाब वो राजपथ ही देगा लेकिन यह सही नहीं है',
 'वेब सिरीज़ तांडव को लेकर जारी विवाद पर आज का कार्टून',
 'बेका समझौता H1B वीज़ा के नए नियम  क्वॉड समूह का गठन - कुछ ऐसे फैसले हैं जो ट्रंप कार्यकाल में अमेरिका-भारत के बीच हुए तो क्या ट्रंप शासन में भारत के साथ सब कुछ अच्छा-अच्छा ही रहा',
 'विपक्षी कांग्रेस पार्टी ने सरकार से स्थिति स्पष्ट करने को कहा है दूसरी ओर सरकार का कहना है कि स्थिति पर उसकी नज़र है',
 'बाइडन-कमला हैरिस की टीम ने अपने प्रशासन के लिए भारतीय मूल के 20 अमेरिकी सदस्यों की

In [18]:
# Converting the list to text files:

with open(r'Data_Approach2\fake_news.txt', 'w', encoding="utf-8") as fp:
    fp.write('\n'.join(news_fake))


with open(r'Data_Approach2\true_news.txt', 'w', encoding="utf-8") as fp:
    fp.write('\n'.join(news_true))


In [21]:

data=pd.read_csv(r'Data_Approach2\fake_news.txt')
data=data.iloc[:,0].tolist()
translator = Translator()
translated=[]
for i in tqdm(data):
    translated_text = translator.translate(i ,src='hi',dest='en')
    translated.append(translated_text.text)
df=pd.DataFrame(translated)
df.to_csv('Translated_fake_news.csv')

100%|██████████| 1250/1250 [19:08<00:00,  1.09it/s]


In [22]:
# Translating the text file to English and saving it in csv file
data=pd.read_csv(r'Data_Approach2\true_news.txt')
data=data.iloc[:,0].tolist()
translator = Translator()
translated=[]
for i in tqdm(data):
    translated_text = translator.translate(i ,src='hi',dest='en')
    translated.append(translated_text.text)
df=pd.DataFrame(translated)
df.to_csv('Translated_true_news.csv')

100%|██████████| 759/759 [17:19<00:00,  1.37s/it]


In [8]:
fake_news = pd.read_csv("Data_Approach2\Translated_fake_news.csv")
fake_news

,short_description
0,Boom found that Imran Khan was criticizing the...
1,Some people of the Sikh community are set to s...
2,Posts are being shared with a large number of ...
3,It is claimed that BJP leaders have admitted t...
4,Media outlets and wire agency incorrectly clai...
...,...
1245,Video of the wrestling match between Indian wr...
1246,"Talking to Boom, the SBI official said that it..."
1247,BJP MP Paresh Rawal has once again lived up to...
1248,"Fake news websites, which suddenly active in t..."


In [9]:
true_news = pd.read_csv("Data_Approach2\Translated_true_news.csv")
true_news

,short_description
0,"In the Brisbane Test, Cheteshwar Pujara, who w..."
1,Pakistani TV presenter Syed Iqarul Hasan came ...
2,Someone should be asked where the first Republ...
3,Today's cartoon on the ongoing controversy ove...
4,The formation of the new rules of the BCA Agre...
...,...
754,The driver continued to drive it inste
755,This missile can kill more than 400 kilometers
756,The center of attraction was the powerful wres...
757,"There is a lot of caste equations of Bihar, bu..."


In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhavy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Process of Stemming

In [5]:
port_stem = PorterStemmer()

In [12]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [19]:
fake_news['short_description'] = fake_news['short_description'].apply(stemming)
true_news['short_description'] = true_news['short_description'].apply(stemming)

In [20]:
fake_news

,short_description
0,boom found imran khan critic govern india curr...
1,peopl sikh commun set set set photograph show ...
2,post share larg number pictur claim social med...
3,claim bjp leader admit govern wrong public
4,media outlet wire agenc incorrectli claim form...
...,...
1245,video wrestl match indian wrestler kavita devi...
1246,talk boom sbi offici said atm brown label mach...
1247,bjp mp paresh rawal live reput spread fake new...
1248,fake news websit suddenli activ month may clos...


In [21]:
true_news

,short_description
0,brisban test cheteshwar pujara call wall india...
1,pakistani tv present sy iqarul hasan came targ...
2,someon ask first republ day parad held delhi a...
3,today cartoon ongo controversi web seri orgi
4,format new rule bca agreement h b visa format ...
...,...
754,driver continu drive inst
755,missil kill kilomet
756,center attract power wrestler relat famou bhol...
757,lot cast equat bihar long time bjp engag mobil...


In [22]:
# Converting them to csv files for applying machine learning model.
true_news.to_csv('Data_Approach2\\clean_true_news.csv')
fake_news.to_csv('Data_Approach2\\clean_fake_news.csv')

In [23]:
# Checking the length of the datasets
len(true_news), len(fake_news)
fake_news = fake_news.head(len(true_news))
len(true_news), len(fake_news)

(759, 759)

In [24]:
# Assigning the labels to process the dataset.
# Fake news = 1
# Valid news = 0

fake_news['label'] = 1
true_news['label'] = 0

C:\Users\bhavy\AppData\Local\Temp\ipykernel_22612\1482499588.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_news['label'] = 1


In [30]:
# Joining true_news and false_news dataset
news = pd.concat([fake_news, true_news])
from sklearn.utils import shuffle
news = shuffle(news)

In [31]:
print("Final Dataset for applying the model: ")
news

Final Dataset for applying the model: 


,short_description,label
483,occa guru parab nanak becam rebelli age import...,0
646,indian armi tweet letter fake agreement viral ...,1
26,world largest vaccin program start india mani ...,0
70,claim today nihang singh left delhi support fa...,1
142,boom found total four pictur shahe e azam bhag...,1
...,...,...
318,suprem court comment peasant movement today ca...,0
517,boom examin profil found account fake accord a...,1
527,claim women given money shaheen bagh boom foun...,1
46,viral pictur big mace bow arrow seen middl squar,1


In [3]:
# Converting the final dataset to a csv file.
news.to_csv('Data_Approach2\\Final_Dataset.csv')

NameError: name 'news' is not defined

In [37]:
#separating the data and label
X = news['short_description'].values
Y = news['label'].values

In [38]:
print(X)
print(Y)

['occa guru parab nanak becam rebelli age import thing guru nanak life'
 'indian armi tweet letter fake agreement viral name bipin rawat'
 'world largest vaccin program start india mani peopl doubt vaccin' ...
 'claim women given money shaheen bagh boom found video distribut februari'
 'viral pictur big mace bow arrow seen middl squar'
 'boom found note shivaji fake america declar februari chhatrapati shivaji day']
[0 1 0 ... 1 1 1]


In [39]:
Y.shape

(1518,)

In [40]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [41]:
print(X)

  (0, 3220)	0.2359293302265176
  (0, 2608)	0.27559767366697696
  (0, 2311)	0.27559767366697696
  (0, 2238)	0.2303633064599518
  (0, 2131)	0.5511953473339539
  (0, 1817)	0.20533536830847482
  (0, 1483)	0.21090139207504063
  (0, 1336)	0.4850250781348726
  (0, 358)	0.22554179736402302
  (0, 75)	0.2505697355155
  (1, 3420)	0.17370983043776853
  (1, 3320)	0.29600746451994703
  (1, 2594)	0.4221439908116651
  (1, 2129)	0.2679953674345587
  (1, 1808)	0.3462522391875436
  (1, 1501)	0.2501578766174087
  (1, 1110)	0.22681704674548853
  (1, 431)	0.4221439908116651
  (1, 214)	0.3031687258390124
  (1, 84)	0.3579735122006338
  (2, 3508)	0.2877739798739167
  (2, 3375)	0.5208633317234336
  (2, 3050)	0.28986979360151505
  (2, 2489)	0.35318120044108126
  (2, 2358)	0.21737080127953687
  :	:
  (1515, 1197)	0.13609686038133312
  (1515, 1138)	0.3540895959402065
  (1515, 945)	0.3810878414662934
  (1515, 651)	0.19269384016028548
  (1515, 459)	0.12195445217770638
  (1515, 298)	0.3902956619970315
  (1516, 3420)	

Splitting the dataset and training the model

In [2]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

NameError: name 'train_test_split' is not defined

In [44]:
model = LogisticRegression()

In [45]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluating the model

In [46]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [47]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9769357495881383


In [48]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [49]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9736842105263158


In [50]:
print('Training data : ', training_data_accuracy*100)
print('Test data : ', test_data_accuracy*100)

Training data :  97.69357495881383
Test data :  97.36842105263158
